In [1]:
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt

In [2]:
def resize_img(image):
    ht = int(image.shape[0]*0.15)
    wd = int(image.shape[1]*0.15)
    image_rs = cv2.resize(image, (wd,ht))
    return image_rs

In [49]:
def getCircles(imagebw):
    height,width = imagebw.shape
    mask = np.zeros((height,width), np.uint8)
    edges = cv2.Canny(imagebw, 100, 200)
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, 10000, param1 = 50, param2 = 30, minRadius = 0, maxRadius = 0)
    return(mask, circles)

In [94]:
def drawContours(img, contours, heir):
    drawing = np.zeros((img.shape[0], img.shape[1], img.shape[2]))
    for i in range(len(contours)):
        x,y,w,h = cv2.boundingRect(contours[i])
        cnt_area = cv2.contourArea(contours[i], False)
        color_contours = (0, 100, 250)
        cv2.drawContours(drawing, contours, i, color_contours, 1, 8, heir,2)
    return(drawing)

In [51]:
img = cv2.imread('Seeds3.jpeg')
img = resize_img(img)
imgbw = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
i24 = cv2.imread('Seeds3_24.jpeg')
i24 = resize_img(i24)
i24bw = cv2.cvtColor(i24, cv2.COLOR_BGR2GRAY)

In [52]:
mask0, circles0 = getCircles(imgbw)
mask1, circles1 = getCircles(i24bw)

if circles0 is not None:
    for i in circles0[0,:]:
        cv2.circle(mask0,(i[0],i[1]),i[2],(255,255,255),thickness=-1)
imgcr1 = img
masked_data0 = cv2.bitwise_and(imgcr1, imgcr1, mask=mask0)

if circles1 is not None:
    for i in circles1[0,:]:
        cv2.circle(mask1,(i[0],i[1]),i[2],(255,255,255),thickness=-1)
imgcr2 = i24
masked_data1 = cv2.bitwise_and(imgcr2, imgcr2, mask=mask1)

In [95]:
hsv = cv2.cvtColor(masked_data0, cv2.COLOR_BGR2HSV)
lower_red = np.array([11,55,143])
upper_red = np.array([31,148,223])
mask1 = cv2.inRange(hsv, lower_red, upper_red)

_,contours0, heir0 = cv2.findContours(mask1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
drawing0 = drawContours(masked_data0, contours0, heir0)

In [103]:
hsv = cv2.cvtColor(masked_data1, cv2.COLOR_BGR2HSV)
lower_yel = np.array([11,55,143])
upper_yel = np.array([31,148,250])
mask21 = cv2.inRange(hsv, lower_yel, upper_yel)
#hsv = cv2.cvtColor(masked_data1, cv2.COLOR_BGR2HSV)
lower_purp = np.array([8,50,20])
upper_purp = np.array([30,180,170])
mask22 = cv2.inRange(hsv, lower_purp, upper_purp)

mask2 = mask21 + mask22

output_img = masked_data1.copy()
output_img[np.where(mask2==0)] = 150

_,contours1, heir1 = cv2.findContours(mask2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
drawing1 = drawContours(masked_data1, contours1, heir1)

In [101]:
cv2.imwrite('Seed1-Ext.png', output_img0)

True

In [99]:
output_img0 = masked_data0.copy()
output_img0[np.where(mask1==0)] = 0


In [104]:
cv2.imshow('Mask', mask1)
cv2.imshow('Extract', output_img)
cv2.imshow('Contour1', drawing1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [105]:
cv2.imwrite('Contour1.png', drawing1)
cv2.imwrite('Mask1.png', mask2)
cv2.imwrite('EXtract1.png', output_img)

True